<a href="https://colab.research.google.com/github/RGK73/Amazon_Vine_Analysis/blob/main/movies_Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-02 10:08:15--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2022-04-02 10:08:17 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [5]:
# Create the vine_table. DataFrame
movies_vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
movies_vine_df.drop_duplicates()
movies_vine_df.printSchema()
movies_vine_df.show()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LI5TRP3YIDQL|          5|            0|          0|   N|                Y|
|R3LGC3EKEG84PX|          5|            0|          0|   N|                Y|
| R9PYL3OYH55QY|          5|            0|          1|   N|                Y|
|R3PWBAWUS4NT0Q|          3|            0|          0|   N|                Y|
|R15LYP3O51UU9E|          5|            0|          0|   N|                Y|
|R1AD7L0CC3DSRI|          5|            0|          0|   N|                Y|
|R32FE8Y45QV434|          5|    

## Step 1
Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [6]:
 # select all rows except review_id
df_reviews = movies_vine_df.select(["star_rating","helpful_votes","total_votes","vine","verified_purchase"])
df_reviews.show()

# filter df_reviews for all rows where total_votes >= 20
df_reviews_20 = df_reviews.filter(df_reviews["total_votes"] >= 20)
df_reviews_20.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          1|   N|                Y|
|          3|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          2|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          4|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

## Step 2
Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

In [7]:
# filter df_reviews_20 for all rows where helpful+_votes/total_votes >= 0.5
df_reviews_50 = df_reviews_20.filter(df_reviews_20["helpful_votes"] / df_reviews_20["total_votes"] >= 0.5)
df_reviews_50.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           25|         26|   N|                Y|
|          5|           25|         26|   N|                N|
|          5|           19|         20|   N|                Y|
|          5|           19|         21|   N|                Y|
|          5|           46|         48|   N|                N|
|          4|           46|         46|   N|                N|
|          5|          292|        300|   N|                N|
|          4|           21|         22|   N|                N|
|          5|           26|         26|   N|                N|
|          2|           11|         22|   N|                N|
|          5|           26|         30|   N|                Y|
|          4|           26|         27|   N|                N|
|          3|           14|         20|   N|           

## Step 3
Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

In [8]:
# filter df_reviews_50 for all rows where vine = 'Y'
paid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "Y")
paid_reviews_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           18|         28|   Y|                N|
|          4|           20|         21|   Y|                N|
|          4|           54|         58|   Y|                N|
|          3|           35|         44|   Y|                N|
|          4|           15|         22|   Y|                N|
|          4|           25|         27|   Y|                N|
|          3|           21|         22|   Y|                N|
+-----------+-------------+-----------+----+-----------------+



In [9]:
# summary statistics paid reviews
paid_reviews_df.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|                 7|                 7|                 7|   7|                7|
|   mean|3.5714285714285716|26.857142857142858|31.714285714285715|null|             null|
| stddev|0.5345224838248488|13.582201238245238|14.032275720807439|null|             null|
|    min|                 3|                15|                21|   Y|                N|
|    max|                 4|                54|                58|   Y|                N|
+-------+------------------+------------------+------------------+----+-----------------+



## Step 4
Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

In [10]:
# filter df_reviews_50 for all rows where vine = 'N'
unpaid_reviews_df = df_reviews_50.filter(df_reviews_50["vine"] == "N")
unpaid_reviews_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          3|           25|         26|   N|                Y|
|          5|           25|         26|   N|                N|
|          5|           19|         20|   N|                Y|
|          5|           19|         21|   N|                Y|
|          5|           46|         48|   N|                N|
|          4|           46|         46|   N|                N|
|          5|          292|        300|   N|                N|
|          4|           21|         22|   N|                N|
|          5|           26|         26|   N|                N|
|          2|           11|         22|   N|                N|
|          5|           26|         30|   N|                Y|
|          4|           26|         27|   N|                N|
|          3|           14|         20|   N|           

In [11]:
# summary statistics unpaid reviews
unpaid_reviews_df.describe().show()

+-------+------------------+------------------+------------------+------+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|  vine|verified_purchase|
+-------+------------------+------------------+------------------+------+-----------------+
|  count|            105979|            105979|            105979|105979|           105979|
|   mean| 4.203021353286972| 34.48632276205663|38.640674095811434|  null|             null|
| stddev|1.2847072310247238|35.853572156291975|38.399905374092576|  null|             null|
|    min|                 1|                10|                20|     N|                N|
|    max|                 5|              2181|              2246|     N|                Y|
+-------+------------------+------------------+------------------+------+-----------------+



## Step 5
Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

In [12]:
# determine the count of paid 5 star reviews
paid_five_star_reviews = paid_reviews_df[paid_reviews_df["star_rating"] == 5].count()
paid_five_star_reviews

0

In [13]:
# determine the count of paid reviews
paid_reviews = paid_reviews_df.count()
paid_reviews

7

In [14]:
# determine the % of paid five star reviews
percentage_paid_5star_reviews = (paid_five_star_reviews / paid_reviews) * 100
percentage_paid_5star_reviews

0.0

In [15]:
# determine the count of unpaid 5 star reviews
unpaid_five_star_reviews = unpaid_reviews_df[unpaid_reviews_df["star_rating"] == 5].count()
unpaid_five_star_reviews

67580

In [16]:
# determine the count of unpaid reviews
unpaid_reviews = unpaid_reviews_df.count()
unpaid_reviews

105979

In [17]:
# determine the % of unpaid five star reviews
percentage_unpaid_5star_reviews = (unpaid_five_star_reviews / unpaid_reviews) * 100
percentage_unpaid_5star_reviews

63.767350135404186